-----
<div> <center> ESPACIO PARA BANNER DE LA MAESTRIA </center> </div>   

-----

# Aplicación de LDA
En este Notebook usaremos datos de comentarios realizados por usuarios de Tripadvisor a restaurantes en Bogotá. El objetivo es aplicar la técnica de LDA para extraer algunos tópicos del corpus.

In [1]:
import os 
import pandas as pd

Comenzamos importando la base de datos

In [2]:
os.listdir("data")

['comentarios_tripadvisor.csv',
 'enlaces_tripadvisor.csv',
 'Scraping_tripadvisor.ipynb']

In [3]:
df = pd.read_csv("data/comentarios_tripadvisor.csv", sep = ";")
df.head()

,nombre,link,puntaje_global,n_comentarios,posicion_relativa,n_restaurantes,lat,lon,excelente,muy_bueno,...,características,usuario,relevancia_usuario,fecha_comentario,titulo_comentario,contenido_comentario,fecha_visita,puntaje,rango_de_precios,dietas_especiales
0,Asadero Cimarron del Llano,/Restaurant_Review-g294074-d10003846-Reviews-A...,"4,0",17.0,83.0,127.0,4.666695,-74.113075,9.0,3.0,...,"Comida para llevar, Servicio de mesa, Reservas...",Marylyn73,4 opiniones,Escribió una opinión el 6 de enero de 2021,Comida recalentada,Fui temprano para evitar aglomeraciones y me t...,enero de 2021,10.0,NaN,NaN
1,Asadero Cimarron del Llano,/Restaurant_Review-g294074-d10003846-Reviews-A...,"4,0",17.0,83.0,127.0,4.666695,-74.113075,9.0,3.0,...,"Comida para llevar, Servicio de mesa, Reservas...",jairoenriquer2020,1 opinión,Escribió una opinión el 10 de marzo de 2020,Mala calidad y costo alto,Mala calidad de la carne no es la mamona tradi...,marzo de 2020,10.0,NaN,NaN
2,Asadero Cimarron del Llano,/Restaurant_Review-g294074-d10003846-Reviews-A...,"4,0",17.0,83.0,127.0,4.666695,-74.113075,9.0,3.0,...,"Comida para llevar, Servicio de mesa, Reservas...",HectorLatorre,55 opiniones,Escribió una opinión el 13 de febrero de 2020,Maravilloso sitio de comida llanera,Este es un restaurante bastante típico y muy b...,enero de 2020,40.0,NaN,NaN
3,Asadero Cimarron del Llano,/Restaurant_Review-g294074-d10003846-Reviews-A...,"4,0",17.0,83.0,127.0,4.666695,-74.113075,9.0,3.0,...,"Comida para llevar, Servicio de mesa, Reservas...",Ingriofercas,1 opinión,Escribió una opinión el 13 de enero de 2020,Buen Restaurante Tipico!!!,"buen servicio, eramos mas o menos 20 personas ...",diciembre de 2019,50.0,NaN,NaN
4,Asadero Cimarron del Llano,/Restaurant_Review-g294074-d10003846-Reviews-A...,"4,0",17.0,83.0,127.0,4.666695,-74.113075,9.0,3.0,...,"Comida para llevar, Servicio de mesa, Reservas...",profet2016,429 opiniones,Escribió una opinión el 12 de octubre de 2019,Un recuerdo de la cultura llanera,"Muy buen ambiente, buena comida y muy buena mú...",octubre de 2019,50.0,NaN,NaN


En total tenemos más de 130 mil comentarios (los cuales representan cada una de las filas del dataframe) y 25 variables que describen el restaurante y el comentario.

In [4]:
df.shape 

(132728, 25)

Primero vamos a quedarnos solo con las columnas que nos interesan: `titulo_comentario` y `contenido_comentario`. Luego las unimos.

In [5]:
# Vamos a reducir el tamaño de nuestra muestra para facilitar le computo de
# los procesos
# df = df.sample(10000, random_state = 666).reset_index(drop = True)

In [6]:
comentarios = df["titulo_comentario"] + " " + df["contenido_comentario"]

Procedemos a preprocesar el texto

In [7]:
import unidecode
import re

# Convertimos la columna en texto
comentarios = comentarios.astype(str)

# Quitamos tildes
comentarios = comentarios.apply(lambda x: unidecode.unidecode(x))

# Quitamos comas, guiones y otros caracteres especiales o signos de puntuación
comentarios = comentarios.apply(lambda x:
    re.sub('[^A-Za-z0-9 ]+', ' ', x))

# Ponemos todo el texto en minúscula 
comentarios = comentarios.str.lower()

# Dejamos todos los espacios sencillos
comentarios = comentarios.apply(lambda x: 
    re.sub('\s+', ' ', x))

# Vamos a eliminar todos los números
comentarios = comentarios.apply(lambda x: re.sub("\d+", "", x))
comentarios = comentarios.apply(lambda x: re.sub('\s+', ' ', x))
comentarios = comentarios.str.strip()

In [8]:
# Eliminamos stopwords
from nltk.corpus import stopwords
stopwords = set(stopwords.words("spanish"))

# Creamos un diccionario de stopwords en español
stopwords = [unidecode.unidecode(i) for i in stopwords]
stopwords = set(stopwords)

# Creamos una función que elimine las palabras presentes en un diccionario
def eliminar_stopwords(texto, diccionario):
    texto = [tok for tok in texto.split(" ") if tok not in diccionario]
    return(texto)

# Aplicamos la función para eliminar los stopwords
comentarios = comentarios.apply(lambda x: 
    eliminar_stopwords(x, stopwords))

In [9]:
# Tokenizamos el texto
import spacy
nlp = spacy.load("es_core_news_sm")

# Esto puede tardar un poco
comentarios = comentarios.apply(lambda x: nlp(" ".join(x)))

In [10]:
# Lemmatizamos el texto
comentarios = comentarios.apply(lambda x: [unidecode.unidecode(tok.lemma_) for tok in x])

In [11]:
# Para aplicar LDA necesitamos construir un diccionario
import gensim.corpora as corpora

# A cada palabra se le asigna un id
diccionario = corpora.Dictionary(comentarios)

diccionario.most_common()[0:10]

[('buen', 86367),
 ('comida', 69954),
 ('excelente', 51889),
 ('lugar', 49806),
 ('servicio', 47345),
 ('restaurante', 44469),
 ('atencion', 35712),
 ('plato', 32503),
 ('mejor', 30294),
 ('ambiente', 29256)]

In [12]:
# Creamos un corpus
textos = comentarios.copy()

In [13]:
# Frequencias 
corpus = [diccionario.doc2bow(texto) for texto in textos]

In [14]:
# Aplicamos LDA
from gensim.models.ldamulticore import LdaMulticore
from pprint import pprint

# Debemos escoger el número de tópicos antes de correr el modelo
n_topicos = 3
# Modelo LDA
lda_model = LdaMulticore(corpus = corpus,
    id2word = diccionario,
    num_topics = n_topicos)
# Mostramos las palabras dentro de los 10 tópicos
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.025*"buen" + 0.024*"excelente" + 0.022*"comida" + 0.022*"servicio" + '
  '0.015*"atencion" + 0.014*"restaurante" + 0.011*"comido" + 0.011*"lugar" + '
  '0.011*"plato" + 0.010*"ambiente"'),
 (1,
  '0.024*"comida" + 0.020*"buen" + 0.015*"servicio" + 0.015*"lugar" + '
  '0.009*"restaurante" + 0.008*"plato" + 0.007*"excelente" + 0.007*"ir" + '
  '0.007*"precio" + 0.006*"bien"'),
 (2,
  '0.036*"buen" + 0.022*"comida" + 0.020*"lugar" + 0.019*"excelente" + '
  '0.018*"restaurante" + 0.014*"atencion" + 0.013*"mejor" + 0.012*"plato" + '
  '0.011*"servicio" + 0.011*"ambiente"')]


In [15]:
# Visualizamos los resultados
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()
LDA_visualization = gensimvis.prepare(lda_model, corpus, diccionario)

C:\Users\User\anaconda3\envs\MIAD\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [16]:
LDA_visualization

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.014603  0.016557       1        1  37.466828
1      0.030063 -0.000312       2        1  35.522960
0     -0.015460 -0.016245       3        1  27.010212, topic_info=            Term          Freq         Total Category  logprob  loglift
175    excelente  48840.000000  48840.000000  Default  30.0000  30.0000
67      servicio  47060.000000  47060.000000  Default  29.0000  29.0000
44        comido  18450.000000  18450.000000  Default  28.0000  28.0000
80      atencion  34588.000000  34588.000000  Default  27.0000  27.0000
183  recomendado  13904.000000  13904.000000  Default  26.0000  26.0000
..           ...           ...           ...      ...      ...      ...
43         bueno   4401.459882  16391.531781   Topic3  -5.2211  -0.0059
207       bogota   4648.776508  18471.299647   Topic3  -5.1664  -0.0707
677  experiencia   3768.566342  14621.324673   Topic3  -5.3763  -0.0468
109        hacer   3330.785986  15172.552004   Topic3  -5.4998  -0.2073
12         pedir   3013.336508  11517.100582   Topic3  -5.6000  -0.0318

[306 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
22258      1  0.890471          aaa
19912      1  0.869068      abundar
19912      2  0.115876      abundar
19912      3  0.057938      abundar
5401       1  0.835065  acompanarla
...      ...       ...          ...
11801      2  0.045805        yepes
11801      3  0.045805        yepes
164        1  0.145586         yuca
164        2  0.733529         yuca
164        3  0.120388         yuca

[564 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

Note que puede seleccionar manualmente cada tema para ver sus términos más frecuentes y/o "relevantes", utilizando diferentes valores del parámetro $\lambda$. Esto puede ayudar cuando intenta asignar un nombre interpretable por humanos o un "significado" a cada tema.

Los valores de lambda ($\lambda$) que están muy cerca de cero mostrarán términos que son más específicos para un tema elegido. Lo que significa que verá términos que son "importantes" para ese tema específico pero no necesariamente "importantes" para todo el corpus.

Los valores de lambda que están muy cerca de uno mostrarán aquellos términos que tienen la relación más alta entre la frecuencia de los términos para ese tema específico y la frecuencia general de los términos del corpus.

In [17]:
# Guardamos la visualización como un html. 
# Es mucho más sencillo interactuar con la gráfica desde el archivo que 
# desde el notebook
pyLDAvis.save_html(LDA_visualization, 'visualizacion_LDA.html')